<a href="https://colab.research.google.com/github/Elisabettatrire/Video_Emotion_Recognition/blob/main/LSTM_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#load the necessary libraries
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import tensorflow as tf
from keras import callbacks
from keras import optimizers
from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.datasets import cifar10
from keras.utils import to_categorical
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from keras.models import Sequential

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
X_train = train_datagen.flow_from_directory(
        '.../TIROCINIO/train_ok4',
        target_size=(150,150),
        #batch_size=32,
        shuffle=False,
        class_mode="categorical"
    )

Found 1032 images belonging to 7 classes.


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
X_test = train_datagen.flow_from_directory(
        '.../TIROCINIO/test_ok4',
        target_size=(150,150),
        #batch_size=32,
        shuffle=False,
        class_mode="categorical"
    )

Found 260 images belonging to 7 classes.


In [ ]:
#import of VGG16 model chosen for feature extraction
import os
from tensorflow import keras
modelg = keras.models.load_model('...TIROCINIO/modello_per_features_3200')
modelg.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [ ]:
#calculate features for the training set
features = modelg.predict(X_train)

print(features)


[[0.         0.         0.         ... 0.         0.31986946 0.        ]
 [0.         0.         0.         ... 0.03594523 0.36866438 0.        ]
 [0.         0.         0.         ... 0.02009813 0.32534355 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.5503353  0.        ]
 [0.0682815  0.         0.         ... 0.         0.6612428  0.        ]
 [0.20241982 0.         0.         ... 0.         0.6738532  0.        ]]


In [ ]:
features.shape

(1032, 8192)

In [ ]:
trainf= features.reshape(258,4,8192)
#train_set

In [ ]:
trainf.shape

(258, 4, 8192)

In [ ]:
#calculate features for the test set

featurestest = modelg.predict(X_test)

print(featurestest)

[[0.10125333 0.         0.         ... 0.06139778 0.637197   0.        ]
 [0.259902   0.         0.         ... 0.14237384 0.6901121  0.        ]
 [0.11936915 0.         0.         ... 0.08533359 0.5789906  0.        ]
 ...
 [0.2866547  0.         0.         ... 0.05090619 0.44047058 0.        ]
 [0.21437454 0.         0.         ... 0.04646188 0.45642054 0.        ]
 [0.27361587 0.         0.         ... 0.         0.34537944 0.        ]]


In [ ]:
featurestest.shape

(260, 8192)

In [ ]:
testf= featurestest.reshape(65,4,8192)
#test_set

In [ ]:
testf.shape

(65, 4, 8192)

In [ ]:
#create one hot encoded labels for test set
w=X_test.labels
c = np.zeros((w.size, w.max()+1))
c[np.arange(w.size),w] = 1
ctest=c.reshape(65,4,7)
ctest = ctest[:,0,:]
#ctest = np.argmax(ctest, axis=-1)

In [ ]:
ctest.shape

(65, 7)

In [ ]:
#create one hot encoded labels for train set
v=X_train.labels
b = np.zeros((v.size, v.max()+1))
b[np.arange(v.size),v] = 1
btrain=b.reshape(258,4,7)
btrain = btrain[:,0,:]

In [ ]:
btrain

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
from keras.layers import LSTM
n_timesteps=4
n_features=8192
n_outputs=7
model = Sequential()

#model.add(Dropout(0.5))
#model.add(LSTM(100))
model.add(LSTM(100, input_shape=( n_timesteps,n_features),  return_sequences=True))
#model.add(Dropout(0.5))
model.add(Flatten())
#model.add(Dense(3200, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(trainf, btrain, validation_data=(testf, ctest), epochs=6)

Epoch 1/6
9/9 [==============================] - 3s 120ms/step - loss: 1.7426 - accuracy: 0.3823 - val_loss: 1.3916 - val_accuracy: 0.5538
Epoch 2/6
9/9 [==============================] - 1s 65ms/step - loss: 1.0011 - accuracy: 0.7166 - val_loss: 1.1492 - val_accuracy: 0.5538
Epoch 3/6
9/9 [==============================] - 1s 68ms/step - loss: 0.7472 - accuracy: 0.7746 - val_loss: 1.1419 - val_accuracy: 0.5846
Epoch 4/6
9/9 [==============================] - 1s 64ms/step - loss: 0.5611 - accuracy: 0.8162 - val_loss: 0.9090 - val_accuracy: 0.7077
Epoch 5/6
9/9 [==============================] - 1s 63ms/step - loss: 0.3902 - accuracy: 0.9232 - val_loss: 0.9397 - val_accuracy: 0.6154
Epoch 6/6
9/9 [==============================] - 1s 65ms/step - loss: 0.2509 - accuracy: 0.9711 - val_loss: 0.9622 - val_accuracy: 0.6462


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report

class_labels = list(X_train.class_indices.keys())   
print(class_labels)
v=X_train.labels
b = np.zeros((v.size, v.max()+1))
b[np.arange(v.size),v] = 1
btrain=b.reshape(258,4,7)
btrain = btrain[:,0,:]
btrain = np.argmax(btrain, axis=-1)
predictions = model.predict(trainf)
predictions = np.argmax(predictions, axis= -1)
#predictions.shape

print(confusion_matrix(btrain, predictions,  labels=None, sample_weight=None, normalize=None))
print()

report = classification_report(btrain, predictions, target_names=class_labels)
print(report)

['1_anger', '2_contempt', '3_disgust', '4_fear', '5_happy', '6_sadness', '7_surprise']
[[36  0  1  0  0  0  0]
 [ 1 11  0  0  1  0  0]
 [ 0  0 47  0  1  0  0]
 [ 0  0  0 16  2  0  0]
 [ 0  0  0  0 55  0  0]
 [ 2  0  0  0  0 20  0]
 [ 0  0  0  0  0  0 65]]

              precision    recall  f1-score   support

     1_anger       0.92      0.97      0.95        37
  2_contempt       1.00      0.85      0.92        13
   3_disgust       0.98      0.98      0.98        48
      4_fear       1.00      0.89      0.94        18
     5_happy       0.93      1.00      0.96        55
   6_sadness       1.00      0.91      0.95        22
  7_surprise       1.00      1.00      1.00        65

    accuracy                           0.97       258
   macro avg       0.98      0.94      0.96       258
weighted avg       0.97      0.97      0.97       258



In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report

class_labels = list(X_test.class_indices.keys())   
print(class_labels)
w=X_test.labels
c = np.zeros((w.size, w.max()+1))
c[np.arange(w.size),w] = 1
ctest=c.reshape(65,4,7)
ctest = ctest[:,0,:]
ctest = np.argmax(ctest, axis=-1)
predictionstest = model.predict(testf)
predictionstest = np.argmax(predictionstest, axis= -1)
#predictions.shape

#Confusion matrix whose i-th row and j-th column entry indicates 
#the number of samples with true label being i-th class and predicted label being j-th class.
print(confusion_matrix(ctest, predictionstest,  labels=None, sample_weight=None, normalize=None))
print()

report = classification_report(ctest, predictionstest, target_names=class_labels)
print(report)

['1_anger', '2_contempt', '3_disgust', '4_fear', '5_happy', '6_sadness', '7_surprise']
[[ 5  0  3  0  0  0  0]
 [ 1  1  0  2  1  0  0]
 [ 0  0 10  0  1  0  0]
 [ 0  0  0  0  5  0  1]
 [ 0  0  0  0 11  0  1]
 [ 1  0  1  1  2  0  1]
 [ 0  1  0  1  0  0 15]]

              precision    recall  f1-score   support

     1_anger       0.71      0.62      0.67         8
  2_contempt       0.50      0.20      0.29         5
   3_disgust       0.71      0.91      0.80        11
      4_fear       0.00      0.00      0.00         6
     5_happy       0.55      0.92      0.69        12
   6_sadness       0.00      0.00      0.00         6
  7_surprise       0.83      0.88      0.86        17

    accuracy                           0.65        65
   macro avg       0.47      0.50      0.47        65
weighted avg       0.57      0.65      0.59        65



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.fit(trainf, btrain, validation_data=(testf, ctest), epochs=5)

Epoch 1/5
9/9 [==============================] - 3s 118ms/step - loss: 1.7794 - accuracy: 0.3384 - val_loss: 1.3221 - val_accuracy: 0.5385
Epoch 2/5
9/9 [==============================] - 1s 68ms/step - loss: 1.0829 - accuracy: 0.5649 - val_loss: 1.2198 - val_accuracy: 0.5846
Epoch 3/5
9/9 [==============================] - 1s 69ms/step - loss: 0.7513 - accuracy: 0.8009 - val_loss: 1.1736 - val_accuracy: 0.5538
Epoch 4/5
9/9 [==============================] - 1s 65ms/step - loss: 0.5630 - accuracy: 0.7788 - val_loss: 0.9669 - val_accuracy: 0.6154
Epoch 5/5
9/9 [==============================] - 1s 65ms/step - loss: 0.4024 - accuracy: 0.9236 - val_loss: 1.0816 - val_accuracy: 0.5846


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report

class_labels = list(X_train.class_indices.keys())   
print(class_labels)
v=X_train.labels
b = np.zeros((v.size, v.max()+1))
b[np.arange(v.size),v] = 1
btrain=b.reshape(258,4,7)
btrain = btrain[:,0,:]
btrain = np.argmax(btrain, axis=-1)
predictions = model.predict(trainf)
predictions = np.argmax(predictions, axis= -1)
#predictions.shape

print(confusion_matrix(btrain, predictions,  labels=None, sample_weight=None, normalize=None))
print()

report = classification_report(btrain, predictions, target_names=class_labels)
print(report)

['1_anger', '2_contempt', '3_disgust', '4_fear', '5_happy', '6_sadness', '7_surprise']
[[31  2  3  0  0  1  0]
 [ 0 12  0  0  0  1  0]
 [ 0  0 48  0  0  0  0]
 [ 0  0  1 15  0  0  2]
 [ 0  0  2  0 53  0  0]
 [ 0  1  1  0  0 20  0]
 [ 0  0  0  0  0  0 65]]

              precision    recall  f1-score   support

     1_anger       1.00      0.84      0.91        37
  2_contempt       0.80      0.92      0.86        13
   3_disgust       0.87      1.00      0.93        48
      4_fear       1.00      0.83      0.91        18
     5_happy       1.00      0.96      0.98        55
   6_sadness       0.91      0.91      0.91        22
  7_surprise       0.97      1.00      0.98        65

    accuracy                           0.95       258
   macro avg       0.94      0.92      0.93       258
weighted avg       0.95      0.95      0.95       258



In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report

class_labels = list(X_test.class_indices.keys())   
print(class_labels)
w=X_test.labels
c = np.zeros((w.size, w.max()+1))
c[np.arange(w.size),w] = 1
ctest=c.reshape(65,4,7)
ctest = ctest[:,0,:]
ctest = np.argmax(ctest, axis=-1)
predictionstest = model.predict(testf)
predictionstest = np.argmax(predictionstest, axis= -1)
#predictions.shape

#Confusion matrix whose i-th row and j-th column entry indicates 
#the number of samples with true label being i-th class and predicted label being j-th class.
print(confusion_matrix(ctest, predictionstest,  labels=None, sample_weight=None, normalize=None))
print()

report = classification_report(ctest, predictionstest, target_names=class_labels)
print(report)

['1_anger', '2_contempt', '3_disgust', '4_fear', '5_happy', '6_sadness', '7_surprise']
[[ 3  0  5  0  0  0  0]
 [ 1  2  0  0  0  1  1]
 [ 0  0 11  0  0  0  0]
 [ 1  0  1  0  2  0  2]
 [ 0  0  4  0  7  0  1]
 [ 1  0  4  0  0  0  1]
 [ 0  1  1  0  0  0 15]]

              precision    recall  f1-score   support

     1_anger       0.50      0.38      0.43         8
  2_contempt       0.67      0.40      0.50         5
   3_disgust       0.42      1.00      0.59        11
      4_fear       0.00      0.00      0.00         6
     5_happy       0.78      0.58      0.67        12
   6_sadness       0.00      0.00      0.00         6
  7_surprise       0.75      0.88      0.81        17

    accuracy                           0.58        65
   macro avg       0.45      0.46      0.43        65
weighted avg       0.52      0.58      0.53        65



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from keras.layers import LSTM
n_timesteps=4
n_features=8192
n_outputs=7
model = Sequential()

#model.add(Dropout(0.5))
#model.add(LSTM(100))
model.add(LSTM(100, input_shape=( n_timesteps,n_features),  return_sequences=True))
model.add(Dropout(0.5))
model.add(Flatten())
#model.add(Dense(3200, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 4, 100)            3317200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 4, 100)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 2807      
Total params: 3,320,007
Trainable params: 3,320,007
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(trainf, btrain, validation_data=(testf, ctest), epochs=20)

Epoch 1/20
9/9 [==============================] - 3s 140ms/step - loss: 1.9029 - accuracy: 0.2817 - val_loss: 1.5163 - val_accuracy: 0.4923
Epoch 2/20
9/9 [==============================] - 1s 82ms/step - loss: 1.3194 - accuracy: 0.5140 - val_loss: 1.3020 - val_accuracy: 0.5077
Epoch 3/20
9/9 [==============================] - 1s 83ms/step - loss: 1.1158 - accuracy: 0.5824 - val_loss: 1.2229 - val_accuracy: 0.5231
Epoch 4/20
9/9 [==============================] - 1s 81ms/step - loss: 1.0232 - accuracy: 0.6404 - val_loss: 1.1727 - val_accuracy: 0.5385
Epoch 5/20
9/9 [==============================] - 1s 84ms/step - loss: 0.9928 - accuracy: 0.5970 - val_loss: 1.1792 - val_accuracy: 0.5385
Epoch 6/20
9/9 [==============================] - 1s 80ms/step - loss: 0.8681 - accuracy: 0.6637 - val_loss: 1.1110 - val_accuracy: 0.5538
Epoch 7/20
9/9 [==============================] - 1s 79ms/step - loss: 0.8010 - accuracy: 0.7181 - val_loss: 1.0833 - val_accuracy: 0.6462
Epoch 8/20
9/9 [==========

In [ ]:
model.fit(trainf, btrain, validation_data=(testf, ctest), epochs=50)

Epoch 1/50
9/9 [==============================] - 3s 155ms/step - loss: 1.9066 - accuracy: 0.2672 - val_loss: 1.4309 - val_accuracy: 0.5385
Epoch 2/50
9/9 [==============================] - 1s 79ms/step - loss: 1.3015 - accuracy: 0.5151 - val_loss: 1.2820 - val_accuracy: 0.5385
Epoch 3/50
9/9 [==============================] - 1s 80ms/step - loss: 0.9592 - accuracy: 0.6619 - val_loss: 1.1306 - val_accuracy: 0.5538
Epoch 4/50
9/9 [==============================] - 1s 78ms/step - loss: 0.8796 - accuracy: 0.7082 - val_loss: 1.2063 - val_accuracy: 0.5538
Epoch 5/50
9/9 [==============================] - 1s 81ms/step - loss: 0.7447 - accuracy: 0.7996 - val_loss: 1.0052 - val_accuracy: 0.6154
Epoch 6/50
9/9 [==============================] - 1s 76ms/step - loss: 0.6119 - accuracy: 0.8476 - val_loss: 0.9835 - val_accuracy: 0.6000
Epoch 7/50
9/9 [==============================] - 1s 80ms/step - loss: 0.4869 - accuracy: 0.8479 - val_loss: 0.8948 - val_accuracy: 0.6923
Epoch 8/50
9/9 [==========

In [ ]:
lstm = Sequential()
lstm.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(4, 8192)))
lstm.add(LSTM(25, activation='relu'))
lstm.add(Dense(20, activation='relu'))
#lstm.add(Dense(10, activation='relu'))
lstm.add(Dense(7))
lstm.compile(optimizer='adam', loss='mse')


lstm.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

In [ ]:
lstm.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 4, 200)            6714400   
_________________________________________________________________
lstm_2 (LSTM)                (None, 25)                22600     
_________________________________________________________________
dense_1 (Dense)              (None, 20)                520       
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 147       
Total params: 6,737,667
Trainable params: 6,737,667
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.fit(trainf, btrain, validation_data=(testf, ctest), epochs=50)

Epoch 1/50
9/9 [==============================] - 4s 204ms/step - loss: 0.1856 - accuracy: 0.1790 - val_loss: 0.1331 - val_accuracy: 0.2615
Epoch 2/50
9/9 [==============================] - 1s 152ms/step - loss: 0.1302 - accuracy: 0.2925 - val_loss: 0.1298 - val_accuracy: 0.3231
Epoch 3/50
9/9 [==============================] - 1s 149ms/step - loss: 0.1177 - accuracy: 0.3075 - val_loss: 0.1225 - val_accuracy: 0.2615
Epoch 4/50
9/9 [==============================] - 1s 153ms/step - loss: 0.1139 - accuracy: 0.2846 - val_loss: 0.1177 - val_accuracy: 0.2615
Epoch 5/50
9/9 [==============================] - 1s 150ms/step - loss: 0.1119 - accuracy: 0.2507 - val_loss: 0.1152 - val_accuracy: 0.3077
Epoch 6/50
9/9 [==============================] - 1s 151ms/step - loss: 0.1064 - accuracy: 0.3486 - val_loss: 0.1168 - val_accuracy: 0.4000
Epoch 7/50
9/9 [==============================] - 1s 153ms/step - loss: 0.1070 - accuracy: 0.4026 - val_loss: 0.1143 - val_accuracy: 0.4000
Epoch 8/50
9/9 [====

In [ ]:
from keras.layers import LSTM
n_timesteps=4
n_features=8192
n_outputs=7
model = Sequential()

#model.add(Dropout(0.5))
#model.add(LSTM(100))
model.add(LSTM(200, input_shape=( n_timesteps,n_features),  return_sequences=True))
#model.add(Dropout(0.5))
model.add(Flatten())
#model.add(Dense(3200, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 4, 200)            6714400   
_________________________________________________________________
flatten_3 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 5607      
Total params: 6,720,007
Trainable params: 6,720,007
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(trainf, btrain, validation_data=(testf, ctest), epochs=50)

Epoch 1/50
9/9 [==============================] - 4s 244ms/step - loss: 1.9017 - accuracy: 0.3429 - val_loss: 1.3494 - val_accuracy: 0.5538
Epoch 2/50
9/9 [==============================] - 1s 150ms/step - loss: 0.9991 - accuracy: 0.6141 - val_loss: 1.1982 - val_accuracy: 0.5538
Epoch 3/50
9/9 [==============================] - 1s 147ms/step - loss: 0.8009 - accuracy: 0.7328 - val_loss: 1.1715 - val_accuracy: 0.6769
Epoch 4/50
9/9 [==============================] - 1s 147ms/step - loss: 0.5675 - accuracy: 0.7967 - val_loss: 1.0721 - val_accuracy: 0.6000
Epoch 5/50
9/9 [==============================] - 1s 144ms/step - loss: 0.4145 - accuracy: 0.8766 - val_loss: 0.8592 - val_accuracy: 0.6923
Epoch 6/50
9/9 [==============================] - 1s 150ms/step - loss: 0.2258 - accuracy: 0.9549 - val_loss: 0.8277 - val_accuracy: 0.6923
Epoch 7/50
9/9 [==============================] - 1s 144ms/step - loss: 0.1400 - accuracy: 0.9727 - val_loss: 1.2346 - val_accuracy: 0.5538
Epoch 8/50
9/9 [====

In [ ]:
from keras.layers import LSTM
n_timesteps=4
n_features=8192
n_outputs=7
model = Sequential()

#model.add(Dropout(0.5))
#model.add(LSTM(100))
model.add(LSTM(20, input_shape=( n_timesteps,n_features),  return_sequences=True))
#model.add(Dropout(0.5))
model.add(Flatten())
#model.add(Dense(3200, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(trainf, btrain, validation_data=(testf, ctest), epochs=50)

Epoch 1/50
9/9 [==============================] - 3s 88ms/step - loss: 1.8870 - accuracy: 0.2666 - val_loss: 1.6615 - val_accuracy: 0.4769
Epoch 2/50
9/9 [==============================] - 0s 29ms/step - loss: 1.5193 - accuracy: 0.5667 - val_loss: 1.3654 - val_accuracy: 0.5538
Epoch 3/50
9/9 [==============================] - 0s 29ms/step - loss: 1.1692 - accuracy: 0.6247 - val_loss: 1.3639 - val_accuracy: 0.5385
Epoch 4/50
9/9 [==============================] - 0s 27ms/step - loss: 0.9390 - accuracy: 0.6982 - val_loss: 1.1072 - val_accuracy: 0.5385
Epoch 5/50
9/9 [==============================] - 0s 28ms/step - loss: 0.7295 - accuracy: 0.7958 - val_loss: 1.1505 - val_accuracy: 0.6154
Epoch 6/50
9/9 [==============================] - 0s 31ms/step - loss: 0.7825 - accuracy: 0.7219 - val_loss: 1.1485 - val_accuracy: 0.5846
Epoch 7/50
9/9 [==============================] - 0s 30ms/step - loss: 0.6518 - accuracy: 0.8638 - val_loss: 1.0810 - val_accuracy: 0.6308
Epoch 8/50
9/9 [===========

In [ ]:
from keras.layers import LSTM
n_timesteps=4
n_features=8192
n_outputs=7
model = Sequential()

#model.add(Dropout(0.5))
#model.add(LSTM(100))
model.add(LSTM(20, input_shape=( n_timesteps,n_features),  return_sequences=True))
model.add(Dropout(0.5))
model.add(Flatten())
#model.add(Dense(3200, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(trainf, btrain, validation_data=(testf, ctest), epochs=50)

Epoch 1/50
9/9 [==============================] - 2s 87ms/step - loss: 1.8945 - accuracy: 0.2518 - val_loss: 1.6431 - val_accuracy: 0.4154
Epoch 2/50
9/9 [==============================] - 0s 28ms/step - loss: 1.4831 - accuracy: 0.4521 - val_loss: 1.5137 - val_accuracy: 0.5077
Epoch 3/50
9/9 [==============================] - 0s 28ms/step - loss: 1.3990 - accuracy: 0.5141 - val_loss: 1.4350 - val_accuracy: 0.5231
Epoch 4/50
9/9 [==============================] - 0s 29ms/step - loss: 1.2029 - accuracy: 0.5931 - val_loss: 1.3664 - val_accuracy: 0.5385
Epoch 5/50
9/9 [==============================] - 0s 29ms/step - loss: 1.2878 - accuracy: 0.5262 - val_loss: 1.3928 - val_accuracy: 0.5846
Epoch 6/50
9/9 [==============================] - 0s 30ms/step - loss: 1.0670 - accuracy: 0.6183 - val_loss: 1.2611 - val_accuracy: 0.6308
Epoch 7/50
9/9 [==============================] - 0s 31ms/step - loss: 1.0105 - accuracy: 0.6449 - val_loss: 1.2499 - val_accuracy: 0.6462
Epoch 8/50
9/9 [===========